In [19]:


import sys
import json
import pickle
from transformers import (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)

In [20]:
sys.path.append('/Users/mahbubcseju/Desktop/projects/vul_detect')

In [21]:
with open("../../data/devign/test.jsonl", 'r') as f:
    data = []
    for  line in f:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(e)


In [22]:
config_class, model_class, tokenizer_class = RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer


In [23]:
tokenizer = tokenizer_class.from_pretrained('microsoft/codebert-base')

In [24]:
explanations = pickle.load(open('explanation_devign.pkl', 'rb'))

In [25]:
explanations[0]

{'token_grad_sentence': TokenSalience(tokens=['<s>', 'int', 'Ġff', '_', 'get', '_', 'wav', '_', 'header', '(', 'AV', 'Format', 'Context', 'Ġ*', 's', ',', 'ĠAV', 'IO', 'Context', 'Ġ*', 'pb', ',', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'ĠAV', 'Cod', 'ec', 'Context', 'Ġ*', 'cod', 'ec', ',', 'Ġint', 'Ġsize', ',', 'Ġint', 'Ġbig', '_', 'end', 'ian', ')', 'Ċ', 'Ċ', '{', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġint', 'Ġid', ';', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġuint', '64', '_', 't', 'Ġbit', 'rate', ';', 'ĊĊ', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġif', 'Ġ(', 'size', 'Ġ<', 'Ġ14', ')', 'Ġ{', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġav', 'priv', '_', 'request', '_', 'sample', '(', 'cod', 'ec', ',', 'Ġ"', 'wav', 'Ġheader', 'Ġsize', 'Ġ<', 'Ġ14', '");', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġ', 'Ġreturn', 'ĠA', 'VER', 'ROR', '_', 'IN', 'VAL', 'ID', 'DATA', ';', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġ}', 'ĊĊ', 'ĊĊ', 'Ġ', 'Ġ', 'Ġ', 'Ġcodec', '->', 'cod', 'ec', '_', 'type', 'Ġ', 'Ġ=', 'ĠA

In [26]:
assert len(explanations) == len(data)

In [27]:
def get_max_match(original, start_index,  tokens):
    lo = len(original)
    lt = len(tokens)
    ind , max_score = 0, lo
    for i in range(start_index, lo-lt+ 1):
        found = True
        for j in range(lt):
            if  original[i + j] != tokens[j]:
                found = False
        if found:
            return [x for x in range(i, i + lt)]
    return []

In [31]:
def calculate_score(scores, ids):
    sum = 0
    for id in ids:
        sum += scores[id]
    return sum

In [32]:
results = []

for sample, explanation in zip(data, explanations):
    # print(sample, explanation)
    original_tokens = ['<s>'] + tokenizer.tokenize(sample['func'])[:510] + ['</s>']
    explanation_tokens = explanation['token_grad_sentence'].tokens
    explanation_scores = explanation['token_grad_sentence'].salience
    if original_tokens != explanation_tokens:
        print(sample['idx'])
    
    lines = sample['func'].split("\n")
    # print(len(lines))
    # while lines.count(''):
    #     lines.remove('')
    # print(lines[0])
    # print(tokenizer.tokenize(lines[0]))
    # print(explanation_tokens)
    # print(explanation_scores)
    # break
    start_index = 0
    lines_scores = []
    cnt = 0
    for line in lines:
        line_tokens = tokenizer.tokenize(line)
        ids = get_max_match(explanation_tokens, start_index, line_tokens)
        if  ids:
            start_index = ids[-1] + 1
            cnt += 1
        # print(line_tokens)
        # print(ids)
        # print(start_index)
        lines_scores.append(calculate_score(explanation_scores, ids))
    # print(len(lines), len(lines_scores))
    # break
    assert cnt > 0
    results.append({
        'idx': sample['idx'],
        'scores': lines_scores,
    })


In [34]:
json.dump(results, open("line_scores.json", 'w'))